# 데이터 수집 - Selenium, BeautifulSoup

In [ ]:
from bs4 import BeautifulSoup  
import pandas as pd

In [4]:
# 1. 영화데이터진흥원에서 다운로드 받은 파일을 불러옴.
df=pd.read_excel('KOBIS_300.xlsx')

In [5]:
df.head()

,순번,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분
0,1,택시운전사,장훈,더램프(주),NaN,(주)쇼박스,2017-08-02,개봉영화,장편,한국,1906,95282655549,12099614,22185947699,2729766,드라마,15세이상관람가,일반영화
1,2,신과함께-죄와 벌,김용화,"리얼라이즈픽쳐스(주),(주)덱스터스튜디오",NaN,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,개봉영화,장편,한국,1912,68350496432,8539495,16324526082,1984373,판타지,12세이상관람가,일반영화
2,3,공조,김성훈,"(주)제이케이필름,씨제이이앤엠(주),영화사 이창,(주)에이치비엔터테인먼트",NaN,씨제이이앤엠(주),2017-01-18,개봉영화,장편,한국,976,63781731326,7817446,14672808676,1744624,액션,15세이상관람가,일반영화
3,4,스파이더맨: 홈 커밍,존 와츠,NaN,소니픽쳐스엔터테인먼트코리아주식회사극장배급지점,소니픽쳐스엔터테인먼트코리아주식회사극장배급지점,2017-07-05,개봉영화,장편,미국,1965,59120173820,7257973,16753103620,1969107,액션,12세이상관람가,일반영화
4,5,범죄도시,강윤성,"(주)홍필름,(주)비에이엔터테인먼트",NaN,"메가박스중앙(주)플러스엠,(주)키위미디어그룹",2017-10-03,개봉영화,장편,한국,1315,56077813249,6851297,14046905449,1670254,범죄,청소년관람불가,일반영화


In [6]:
# 변수로 사용하지 않을 변수 삭제

df1=df.drop(['감독', '제작사', '수입사', '배급사', '영화유형', '영화형태', '전국 스크린수', '서울 매출액', '서울 관객수', '영화구분'], axis=1)

In [129]:
import time
from selenium import webdriver
import xlrd
from xlutils.copy import copy
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
import os

### 데이터 수집1.
- 영화 네티즌평점, 전문가 평점, 청중 평점 수집

In [187]:
def getmovieinfo_1():
    driver=webdriver.Chrome(r'C:\Users\ehdry\OneDrive\2019_Spring\1. web scraping\chromedriver')
    driver.get("https://movie.naver.com/movie/running/current.nhn")
    name1 = df1['영화명']
    #전체 상영 영화 페이지
    audratelist=[]
    netratelist=[]
    proratelist=[]
    videonumlist=[]
    for i in range(300):
        name = df1['영화명'][i]
        keyword = str(name) #숫자인 경우가 있어서 str()
    
        elem = driver.find_element_by_id("ipt_tx_srch")
        elem.send_keys(keyword)    

        elem = driver.find_element_by_xpath('//*[@id="jSearchArea"]/div/button')
        elem.click()        
    #가장 위의 검색결과 검색
        elem = driver.find_element_by_xpath('//*[@id="old_content"]/ul[2]/li[1]/dl/dt/a')
        elem.click() 
        
        
        try: 
            elem=driver.find_element_by_class_name("score")
            elem=elem.find_element_by_class_name("st_on")
            elem=elem.get_attribute("style")
            print(elem)
    #float(elem[-6:-2])
            print(float(re.search("\d+(\.\d+)?",elem).group())/10)
       
            audrate=float(re.search("\d+(\.\d+)?",elem).group())/10
            if audrate==0:
                audrate=None
        except:
            audrate=None
        audratelist.append(audrate)

        ###전문가 평점
        try:    
            elem=driver.find_element_by_class_name("spc_score_area")
            elem=elem.find_element_by_class_name("st_on")
            elem=elem.get_attribute("style")
            prorate=float(re.search("\d+(\.\d+)?",elem).group())/10
            if prorate==0:
                prorate=None
        except:
            prorate=None
        proratelist.append(prorate)    

        ###네티즌 평점
        try:
            elem=driver.find_element_by_id("pointNetizenPersentBasic")
            elem=elem.find_element_by_class_name("st_on")
            elem=elem.get_attribute("style")
            netrate=float(re.search("\d+(\.\d+)?",elem).group())/10
            if netrate==0:
                netrate=None
        except:
            netrate=None
        netratelist.append(netrate)
        
        
        
    #개별 영화 페이지

        #time.sleep(0.5)
    name1=df['영화명'][0:300]
    infomain_try=pd.DataFrame({"Movie":name1,"AudienceRating":audratelist,"NetizenRating":netratelist,"ExpertRating":proratelist})
    driver.close()
    infomain_try.to_csv("infomain.csv", index=False, encoding="euckr")
  
    return(None)

In [188]:
getmovieinfo_1()

width: 92.8%;
9.28
width: 87.3%;
8.73
width: 88.3%;
8.83
width: 88.3%;
8.83
width: 92.8%;
9.28
width: 73.4%;
7.340000000000001
width: 90.3%;
9.03
width: 89.9%;
8.99
width: 86.1%;
8.61
width: 90.2%;
9.02
width: 83.2%;
8.32
width: 87.3%;
8.73
width: 81.7%;
8.17
width: 91.8%;
9.18
width: 90.2%;
9.02
width: 87.6%;
8.76
width: 92.3%;
9.23
width: 85.8%;
8.58
width: 82.5%;
8.25
width: 86.2%;
8.620000000000001
width: 90.5%;
9.05
width: 81.7%;
8.17
width: 69%;
6.9
width: 85.6%;
8.559999999999999
width: 88.4%;
8.84
width: 87%;
8.7
width: 84.8%;
8.48
width: 92.2%;
9.22
width: 82.4%;
8.24
width: 84.5%;
8.45
width: 89.3%;
8.93
width: 81.5%;
8.15
width: 93.1%;
9.309999999999999
width: 75.9%;
7.590000000000001
width: 94.9%;
9.49
width: 79%;
7.9
width: 91%;
9.1
width: 81.2%;
8.120000000000001
width: 79.4%;
7.94
width: 80.5%;
8.05
width: 78.3%;
7.83
width: 84.1%;
8.41
width: 83.9%;
8.39
width: 68.7%;
6.87
width: 77.4%;
7.74
width: 81.8%;
8.18
width: 74.2%;
7.42
width: 78.2%;
7.82
width: 66.4%;
6.640000

### 데이터 수집2.
- 동영상 수 

In [59]:

from selenium import webdriver
import time
def getmovieinfo_2():
    driver=webdriver.Chrome(r'C:\Users\ehdry\OneDrive\2019_Spring\1. web scraping\chromedriver')
    
    driver.get('http://movie.naver.com/movie/running/current.nhn')
    driver.find_element_by_xpath('//*[@id="gnb_login_button"]/span[3]').click()
    id = 'ehdry3939'
    pw = 'j23118790!$&@'
    driver.execute_script("document.getElementsByName('id')[0].value=\'" + id + "\'")
    time.sleep(1)
    driver.execute_script("document.getElementsByName('pw')[0].value=\'" + pw + "\'")
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
    time.sleep(0.5)
    driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/span[2]/a').click()
    
    #driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/span[2]/a').click()
   
    #options = webdriver.ChromeOptions()
    #options.add_argument('disable-infobars')
    #options.add_argument("User-Agent={Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36}")
    #options.add_argument('--user-data-dir={%LOCALAPPDATA%\Google\Chrome\User Data}')  # 추가
    #driver = webdriver.Remote('http://localhost:4444/wd/hub', options.to_capabilities())  # 셀레 서버를 Standalone 방식으로 가동
    #driver.get('https://nid.naver.com/nidlogin.login')
    #driver.implicitly_wait(random.randrange(10, 16))  # 크롬에 저장된 로그인 정보 가져오는 시간 벌어주는 용도 (10 ~ 15초)
    #driver.find_element_by_css_selector("#frmNIDLogin > fieldset > input").click()
    
    name1 = df1['영화명']
    #전체 상영 영화 페이지
   # audratelist=[]
    #netratelist=[]
    videonumlist=[]
    for i in range(300):
        name = df1['영화명'][i]
        keyword = str(name) #숫자인 경우가 있어서 str()
    
        elem = driver.find_element_by_id("ipt_tx_srch")
        elem.send_keys(keyword)    

        elem = driver.find_element_by_xpath('//*[@id="jSearchArea"]/div/button')
        elem.click()        
    #가장 위의 검색결과 검색
        elem = driver.find_element_by_xpath('//*[@id="old_content"]/ul[2]/li[1]/dl/dt/a')
        elem.click() 
        
        
        try: 
            elem = driver.find_element_by_xpath('//*[@id="movieEndTabMenu"]/li[4]/a')
            elem.click()
        
            elem = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[4]/div/div[1]/div/div/a[2]')
            elem.click() 
            source = driver.page_source
            soup = BeautifulSoup(source, 'lxml')
            videonum = soup.find(class_="_totalCount").string
            #elem=driver.find_element_by_class_name("_totalCount")
            #elem=elem.find_element_by_class_name("st_on")
            #elem=elem.get_attribute("style")
            
    #float(elem[-6:-2])
            if videonum==0:
                videonum=None
        except:
            videonum=None
        videonumlist.append(videonum)

        #time.sleep(0.5)
    name1=df['영화명'][0:300]
    infomain_try=pd.DataFrame({"Movie":name1,"VideoNumber":videonumlist})
    driver.close()
    infomain_try.to_csv("infomain_2.csv", index=False, encoding="euckr")
  
    return(None)

In [60]:
getmovieinfo_2()

In [31]:
from selenium import webdriver
def getmovieinfo_2():
    driver=webdriver.Chrome(r'C:\Users\ehdry\OneDrive\2019_Spring\1. web scraping\chromedriver')
    driver.get("https://movie.naver.com/movie/running/current.nhn")
    
    
    name1 = df1['영화명']
    #전체 상영 영화 페이지
   # audratelist=[]
    #netratelist=[]
    videonum_list=[]
    for i in range(300):
        name = df1['영화명'][i]
        keyword = str(name) #숫자인 경우가 있어서 str()
    
        elem = driver.find_element_by_id("ipt_tx_srch")
        elem.send_keys(keyword)    

        elem = driver.find_element_by_xpath('//*[@id="jSearchArea"]/div/button')
        elem.click()        
    #가장 위의 검색결과 검색
        elem = driver.find_element_by_xpath('//*[@id="old_content"]/ul[2]/li[1]/dl/dt/a')
        elem.click() 
        
        
        try: 
            elem = driver.find_element_by_xpath('//*[@id="movieEndTabMenu"]/li[4]/a')
            elem.click()
        
            elem = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div[4]/div/div[1]/div/div/a[2]')
            elem.click() 
            source = driver.page_source
            soup = BeautifulSoup(source, 'lxml')
            videonum = soup.find(class_="_totalCount").string
            #elem=driver.find_element_by_class_name("_totalCount")
            #elem=elem.find_element_by_class_name("st_on")
            #elem=elem.get_attribute("style")
            
    #float(elem[-6:-2])
            if videonum==0:
                videonum=None
        except:
            videonum=None
        videonum_list.append(videonum)

        #time.sleep(0.5)
    name1=df['영화명'][0:300]
    infomain_try=pd.DataFrame({"Movie":name1,"VideoNumber":videonum_list})
    driver.close()
    infomain_try.to_csv("infomain_2.csv", index=False, encoding="euckr")
  
    return(None)

In [10]:
import pandas as pd
dataset = pd.read_csv('infomain.csv',encoding='CP949')

In [11]:
dataset.head()

,Movie,AudienceRating,NetizenRating,ExpertRating
0,택시운전사,9.28,9.04,6.09
1,신과함께-죄와 벌,8.73,7.83,5.92
2,공조,8.83,8.27,5.00
3,스파이더맨: 홈 커밍,8.83,8.37,7.00
4,범죄도시,9.28,9.14,6.00


In [12]:
dataset1 = pd.read_csv('infomain_2.csv',encoding='CP949')

In [13]:
dataset1.head()

,Movie,VideoNumber
0,택시운전사,20.0
1,신과함께-죄와 벌,17.0
2,공조,17.0
3,스파이더맨: 홈 커밍,8.0
4,범죄도시,14.0


In [14]:
df1=pd.merge(dataset, dataset1)
df1.head()

,Movie,AudienceRating,NetizenRating,ExpertRating,VideoNumber
0,택시운전사,9.28,9.04,6.09,20.0
1,신과함께-죄와 벌,8.73,7.83,5.92,17.0
2,공조,8.83,8.27,5.00,17.0
3,스파이더맨: 홈 커밍,8.83,8.37,7.00,8.0
4,범죄도시,9.28,9.14,6.00,14.0


In [15]:
df2=df.drop(['감독', '제작사', '수입사', '배급사', '영화유형', '영화형태', '전국 스크린수', '서울 매출액', '서울 관객수', '영화구분'], axis=1)

In [16]:
df2.head()

,순번,영화명,개봉일,국적,전국 매출액,전국 관객수,장르,등급
0,1,택시운전사,2017-08-02,한국,95282655549,12099614,드라마,15세이상관람가
1,2,신과함께-죄와 벌,2017-12-20,한국,68350496432,8539495,판타지,12세이상관람가
2,3,공조,2017-01-18,한국,63781731326,7817446,액션,15세이상관람가
3,4,스파이더맨: 홈 커밍,2017-07-05,미국,59120173820,7257973,액션,12세이상관람가
4,5,범죄도시,2017-10-03,한국,56077813249,6851297,범죄,청소년관람불가


In [17]:
df2 = df.rename(columns = {'영화명': 'Movie'})

In [18]:
df3=pd.merge(df2, df1)

In [19]:
df4 = df3.drop(['제작사', '수입사', '배급사', '영화유형', '영화형태', '전국 스크린수', '서울 매출액', '서울 관객수', '영화구분'], axis=1)

In [20]:
dataset2 = pd.read_csv('infomain_4.csv',encoding='CP949')
df5=pd.merge(df4, dataset2)
df5.head()
df5.to_csv("movie_information.csv", mode="w",encoding='ms949')

1) 네이버 네티즌 평점 
2) 네이버 기자/평론가 평점
3) 네이버 동영상 수
4) 개봉일: year, month, weekday (요일)
5) 제작국가
6) 장르
7) 관람등급
8) 상영시간


In [ ]:
from urllib.request import urlopen
html = urlopen('https://movie.naver.com/movie/bi/mi/basic.nhn?code=137690')
soup = BeautifulSoup(html, 'lxml')

In [ ]:
            #elem=driver.find_element_by_id("pointNetizenPersentBasic")
            #elem=elem.find_element_by_class_name("st_on")
            #elem=elem.get_attribute("style")
            #netrate=float(re.search("\d+(\.\d+)?",elem).group())/10
from urllib.request import urlopen
html = urlopen('https://movie.naver.com/movie/bi/mi/basic.nhn?code=137690')
soup = BeautifulSoup(html, 'html5lib')
import re
s = soup.find('p').text
reg = re.search('\d{2,3}분', s).group()
#Reg.search(‘321분)
reg

In [ ]:
from bs4 import BeautifulSoup  
from urllib.request import urlopen
import re
import requests
def getmovieinfo_4():
    driver=webdriver.Chrome(r'C:\Users\ehdry\OneDrive\2019_Spring\1. web scraping\chromedriver')
    driver.get("https://movie.naver.com/movie/running/current.nhn")
    name1 = df1['Movie']
    #전체 상영 영화 페이지
    timelist=[]
    for i in range(300):
        name = df1['Movie'][i]
        keyword = str(name) #숫자인 경우가 있어서 str()
    
        elem = driver.find_element_by_id("ipt_tx_srch")
        elem.send_keys(keyword)    

        elem = driver.find_element_by_xpath('//*[@id="jSearchArea"]/div/button')
        elem.click()        
    #가장 위의 검색결과 검색
        elem = driver.find_element_by_xpath('//*[@id="old_content"]/ul[2]/li[1]/dl/dt/a')
        elem.click() 
        
        current_url=driver.current_url
        html=requests.get(current_url)
        soup = BeautifulSoup(html.text, 'lxml')
        
        ###상영시간
        try:
       
            current_url=driver.current_url
            html=requests.get(current_url)
            soup = BeautifulSoup(html.text, 'lxml')            
            
            
        #current_url = driver.current_url
        #html = requests.get(current_url)
        #soup = BeautifulSoup(html, 'html5lib')
            s = soup.find('p').text
            time = re.search('\d{2,3}분', s).group()
           
        #source = driver.page_source
        #soup = BeautifulSoup(source, 'lxml')
        #html = soup.find("p").text
        #time = re.search('\d{2,3}분', html).group()
            #time=driver.find_element_by_tag_name("p")
            #time=re.search('\d{2,3}분', elem).group()
            
            if time==0:
                time=None
        except:
            time=None
        timelist.append(time)
    print(timelist)
        
        
    #개별 영화 페이지

        #time.sleep(0.5)
    name1=df1['Movie'][0:300]
    infomain_try=pd.DataFrame({"Movie":name1,"RunTime":timelist})
    driver.close()
    infomain_try.to_csv("infomain_4.csv", index=False, encoding="euckr")
  
    return(None)

In [9]:
getmovieinfo_4()

NameError: name 'getmovieinfo_4' is not defined

### 데이터 수집3.
- 상영 시간

In [11]:
from urllib.request import urlopen
import re
import requests
def getmovieinfo_4():
    driver=webdriver.Chrome(r'C:\Users\ehdry\OneDrive\2019_Spring\1. web scraping\chromedriver')
    driver.get("https://movie.naver.com/movie/running/current.nhn")
    name1 = df1['Movie']
    #전체 상영 영화 페이지
    timelist=[]
    for i in range(2):
        name = df1['Movie'][i]
        keyword = str(name) #숫자인 경우가 있어서 str()
    
        elem = driver.find_element_by_id("ipt_tx_srch")
        elem.send_keys(keyword)    

        elem = driver.find_element_by_xpath('//*[@id="jSearchArea"]/div/button')
        elem.click()        
    #가장 위의 검색결과 검색
        elem = driver.find_element_by_xpath('//*[@id="old_content"]/ul[2]/li[1]/dl/dt/a')
        elem.click() 
        
        #current_url=driver.current_url
        #html=requests.get(current_url)
        #soup = BeautifulSoup(html, 'html5lib')
        
        ###상영시간
        #try:
        #current_url = driver.current_url
        #html = requests.get(current_url)
        #soup = BeautifulSoup(html, 'html5lib')
        #s = soup.find('p').text
        #time = re.search('\d{2,3}분', s).group()
           
        source = driver.page_source
        soup = BeautifulSoup(source, 'lxml')
        html = soup.find("p").text
        time = re.search('\d{2,3}분', html).group()
            #time=driver.find_element_by_tag_name("p")
            #time=re.search('\d{2,3}분', elem).group()
            
            #if time==0:
        #        time=None
        #except:
         #   time=None
        timelist.append(time)
    print(timelist)
        
        
    #개별 영화 페이지

        #time.sleep(0.5)
    name1=df1['Movie'][0:2]
    infomain_try=pd.DataFrame({"Movie":name1,"RunTime":timelist})
    driver.close()
    infomain_try.to_csv("infomain_4.csv", index=False, encoding="euckr")
  
    return(None)

1) 네이버 네티즌 평점 2) 네이버 기자/평론가 평점 3) 네이버 동영상 수 4) 개봉일: year, month, weekday (요일) 5) 제작국가 6) 장르 7) 관람등급 8) 상영시간

In [5]:
import pandas as pd
#df5.head()
#df6 = df5.drop(["감독", "전국 매출액", "AudienceRating"], axis=1)
df6 = pd.read_csv('movie_information2.csv',encoding="euckr")

In [6]:
df6.head()

,Unnamed: 0,순번,Movie,개봉일,국적,전국 관객수,장르,등급,NetizenRating,ExpertRating,VideoNumber,RunTime
0,0,1,택시운전사,2017-08-02,한국,12099614,드라마,15세이상관람가,9.04,6.09,20.0,137분
1,1,2,신과함께-죄와 벌,2017-12-20,한국,8539495,판타지,12세이상관람가,7.83,5.92,17.0,139분
2,2,3,공조,2017-01-18,한국,7817446,액션,15세이상관람가,8.27,5.00,17.0,125분
3,3,4,스파이더맨: 홈 커밍,2017-07-05,미국,7257973,액션,12세이상관람가,8.37,7.00,8.0,133분
4,4,5,범죄도시,2017-10-03,한국,6851297,범죄,청소년관람불가,9.14,6.00,14.0,121분


In [23]:
df6['국적'].nunique()

24

In [24]:
df6.describe()

,순번,전국 관객수,NetizenRating,ExpertRating,VideoNumber
count,299.000000,2.990000e+02,286.000000,233.000000,286.000000
mean,150.882943,7.010333e+05,7.902483,5.874721,7.622378
std,86.637812,1.507642e+06,1.046534,1.175193,5.219806
min,1.000000,1.188700e+04,3.810000,2.250000,1.000000
25%,76.500000,2.924700e+04,7.282500,5.200000,4.000000
50%,151.000000,9.475000e+04,8.070000,6.000000,6.000000
75%,225.500000,4.727070e+05,8.710000,6.670000,10.000000
max,300.000000,1.209961e+07,9.440000,8.550000,29.000000


In [25]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 298
Data columns (total 11 columns):
순번               299 non-null int64
Movie            299 non-null object
개봉일              299 non-null datetime64[ns]
국적               299 non-null object
전국 관객수           299 non-null int64
장르               299 non-null object
등급               299 non-null object
NetizenRating    286 non-null float64
ExpertRating     233 non-null float64
VideoNumber      286 non-null float64
RunTime          295 non-null object
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 28.0+ KB


### 더미 변수 만들기

In [26]:
country_dummies = pd.get_dummies(df6['국적'], prefix='국적')

In [27]:
country_dummies

,국적_그리스,국적_노르웨이,국적_뉴질랜드,국적_덴마크,국적_독일,국적_러시아,국적_말레이시아,국적_멕시코,국적_미국,국적_벨기에,...,국적_인도,국적_일본,국적_중국,국적_칠레,국적_캐나다,국적_태국,국적_프랑스,국적_한국,국적_호주,국적_홍콩
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
data = df6.join([country_dummies])

In [29]:
data.head()

,순번,Movie,개봉일,국적,전국 관객수,장르,등급,NetizenRating,ExpertRating,VideoNumber,...,국적_인도,국적_일본,국적_중국,국적_칠레,국적_캐나다,국적_태국,국적_프랑스,국적_한국,국적_호주,국적_홍콩
0,1,택시운전사,2017-08-02,한국,12099614,드라마,15세이상관람가,9.04,6.09,20.0,...,0,0,0,0,0,0,0,1,0,0
1,2,신과함께-죄와 벌,2017-12-20,한국,8539495,판타지,12세이상관람가,7.83,5.92,17.0,...,0,0,0,0,0,0,0,1,0,0
2,3,공조,2017-01-18,한국,7817446,액션,15세이상관람가,8.27,5.00,17.0,...,0,0,0,0,0,0,0,1,0,0
3,4,스파이더맨: 홈 커밍,2017-07-05,미국,7257973,액션,12세이상관람가,8.37,7.00,8.0,...,0,0,0,0,0,0,0,0,0,0
4,5,범죄도시,2017-10-03,한국,6851297,범죄,청소년관람불가,9.14,6.00,14.0,...,0,0,0,0,0,0,0,1,0,0


In [30]:
data1=data.drop(["국적_인도", "국적_일본", "국적_중국", "국적_칠레", "국적_캐나다", "국적_태국", "국적_프랑스", "국적_호주", "국적_홍콩", "국적_러시아", "국적_말레이시아", "국적_멕시코", "국적_벨기에", "국적_브라질", "국적_스페인", "국적_아일랜드", "국적_영국", "국적_그리스", "국적_노르웨이", "국적_뉴질랜드", "국적_덴마크", "국적_독일", "국적_미국"], axis=1)

In [31]:
data1.head()

,순번,Movie,개봉일,국적,전국 관객수,장르,등급,NetizenRating,ExpertRating,VideoNumber,RunTime,국적_한국
0,1,택시운전사,2017-08-02,한국,12099614,드라마,15세이상관람가,9.04,6.09,20.0,137분,1
1,2,신과함께-죄와 벌,2017-12-20,한국,8539495,판타지,12세이상관람가,7.83,5.92,17.0,139분,1
2,3,공조,2017-01-18,한국,7817446,액션,15세이상관람가,8.27,5.00,17.0,125분,1
3,4,스파이더맨: 홈 커밍,2017-07-05,미국,7257973,액션,12세이상관람가,8.37,7.00,8.0,133분,0
4,5,범죄도시,2017-10-03,한국,6851297,범죄,청소년관람불가,9.14,6.00,14.0,121분,1


In [32]:
# 전문가 평점 평균으로 채우기
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 298
Data columns (total 12 columns):
순번               299 non-null int64
Movie            299 non-null object
개봉일              299 non-null datetime64[ns]
국적               299 non-null object
전국 관객수           299 non-null int64
장르               299 non-null object
등급               299 non-null object
NetizenRating    286 non-null float64
ExpertRating     233 non-null float64
VideoNumber      286 non-null float64
RunTime          295 non-null object
국적_한국            299 non-null uint8
dtypes: datetime64[ns](1), float64(3), int64(2), object(5), uint8(1)
memory usage: 38.3+ KB


In [34]:
import datetime
data1["개봉일"]=pd.to_datetime(data1["개봉일"])
data1["day_of_week"] = data1["개봉일"].dt.day_name()

In [40]:
data1["day_of_week"].unique()

array(['Wednesday', 'Tuesday', 'Thursday', 'Friday', 'Saturday'],
      dtype=object)

In [46]:
data1["day_of_week"][0] = data1["day_of_week"][0].replace('Wednesday', '1')
data1["day_of_week"][0] = data


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


'1'

In [59]:
data1["day_of_week"].unique()

array(['1', '0', '2', '3', '4'], dtype=object)

In [146]:
data1.head()

,순번,Movie,개봉일,국적,전국 관객수,장르,NetizenRating,ExpertRating,VideoNumber,RunTime,국적_한국,day_of_week
0,1,택시운전사,2017-08-02,한국,12099614,드라마,9.04,6.09,20.0,137,1,1
1,2,신과함께-죄와 벌,2017-12-20,한국,8539495,판타지,7.83,5.92,17.0,,1,0
2,3,공조,2017-01-18,한국,7817446,액션,8.27,5.00,17.0,125,1,0
3,4,스파이더맨: 홈 커밍,2017-07-05,미국,7257973,액션,8.37,7.00,8.0,133,0,0
4,5,범죄도시,2017-10-03,한국,6851297,범죄,9.14,6.00,14.0,121,1,0


In [72]:
data1["등급"].unique()

array([nan])

In [81]:
del data1["등급"]

In [75]:
data1["day_of_week"]

0      1
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      2
9      0
10     0
11     0
12     2
13     0
14     0
15     0
16     0
17     0
18     2
19     0
20     2
21     2
22     0
23     0
24     0
25     0
26     2
27     0
28     0
29     0
      ..
269    2
270    2
271    0
272    2
273    0
274    2
275    2
276    2
277    2
278    2
279    2
280    2
281    0
282    2
283    2
284    0
285    2
286    2
287    2
288    2
289    2
290    2
291    2
292    2
293    2
294    2
295    0
296    2
297    2
298    2
Name: day_of_week, Length: 299, dtype: int64

In [130]:
data1.head()

,순번,Movie,개봉일,국적,전국 관객수,장르,NetizenRating,ExpertRating,VideoNumber,RunTime,국적_한국,day_of_week
0,1,택시운전사,2017-08-02,한국,12099614,드라마,9.04,6.09,20.0,137,1,1
1,2,신과함께-죄와 벌,2017-12-20,한국,8539495,판타지,7.83,5.92,17.0,,1,0
2,3,공조,2017-01-18,한국,7817446,액션,8.27,5.00,17.0,125,1,0
3,4,스파이더맨: 홈 커밍,2017-07-05,미국,7257973,액션,8.37,7.00,8.0,133,0,0
4,5,범죄도시,2017-10-03,한국,6851297,범죄,9.14,6.00,14.0,121,1,0


In [131]:
df6.head()
df8 = df6.drop(["국적", "전국 관객수", "장르"], axis=1)

In [132]:
data2 = data1.append(df8)

In [138]:
data1.to_csv('201904162021.csv',encoding="euckr")
                     

In [136]:
df8.head()

,순번,Movie,등급,NetizenRating,ExpertRating,VideoNumber,RunTime
0,1,택시운전사,15세이상관람가,9.04,6.09,20.0,137분
1,2,신과함께-죄와 벌,12세이상관람가,7.83,5.92,17.0,139분
2,3,공조,15세이상관람가,8.27,5.00,17.0,125분
3,4,스파이더맨: 홈 커밍,12세이상관람가,8.37,7.00,8.0,133분
4,5,범죄도시,청소년관람불가,9.14,6.00,14.0,121분


In [141]:
data1.head()
# 변수 숫자형으로 바꾸는 함수 : df.ID = pd.to_numeric(df.ID, errors='coerce').fillna(0).astype(np.int64)

,순번,Movie,개봉일,국적,전국 관객수,장르,NetizenRating,ExpertRating,VideoNumber,RunTime,국적_한국,day_of_week
0,1,택시운전사,2017-08-02,한국,12099614,드라마,9.04,6.09,20.0,137,1,1
1,2,신과함께-죄와 벌,2017-12-20,한국,8539495,판타지,7.83,5.92,17.0,,1,0
2,3,공조,2017-01-18,한국,7817446,액션,8.27,5.00,17.0,125,1,0
3,4,스파이더맨: 홈 커밍,2017-07-05,미국,7257973,액션,8.37,7.00,8.0,133,0,0
4,5,범죄도시,2017-10-03,한국,6851297,범죄,9.14,6.00,14.0,121,1,0


In [145]:
df8.head()

,순번,Movie,등급,NetizenRating,ExpertRating,VideoNumber,RunTime
0,1,택시운전사,15세이상관람가,9.04,6.09,20.0,137분
1,2,신과함께-죄와 벌,12세이상관람가,7.83,5.92,17.0,139분
2,3,공조,15세이상관람가,8.27,5.00,17.0,125분
3,4,스파이더맨: 홈 커밍,12세이상관람가,8.37,7.00,8.0,133분
4,5,범죄도시,청소년관람불가,9.14,6.00,14.0,121분


In [128]:
df9["등급"].unique()

array(['15세이상관람가', '12세이상관람가', '청소년관람불가', '전체관람가'], dtype=object)

In [140]:
df9.head()

,순번,Movie,개봉일,국적,전국 관객수,장르,NetizenRating,ExpertRating,VideoNumber,RunTime,국적_한국,day_of_week,등급
0,53,하루,2017-06-15,한국,1129018,스릴러,NaN,NaN,NaN,NaN,1,2,15세이상관람가
1,54,지오스톰,2017-10-19,미국,1028384,액션,7.45,5.50,5.0,109분,0,2,12세이상관람가
2,55,스타워즈: 라스트 제다이,2017-12-14,미국,946547,액션,6.56,7.71,16.0,152분,0,2,12세이상관람가
3,56,불한당: 나쁜 놈들의 세상,2017-05-17,한국,939535,범죄,6.23,5.83,16.0,120분,1,0,청소년관람불가
4,57,그것,2017-09-06,미국,888386,공포(호러),6.95,6.60,4.0,135분,0,0,15세이상관람가


In [106]:
df9["등급"]= pd.to_numeric(df9["등급"], errors='coerce')

In [107]:
df9["등급"]

0      2
1      1
2      2
3      1
4      3
5      2
6      2
7      2
8      0
9      3
10     1
11     2
12     2
13     2
14     2
15     2
16     1
17     0
18     1
19     1
20     3
21     1
22     1
23     2
24     1
25     2
26     2
27     0
28     2
29     1
      ..
269    1
270    2
271    0
272    1
273    1
274    1
275    2
276    1
277    0
278    2
279    2
280    0
281    0
282    2
283    0
284    2
285    1
286    0
287    3
288    0
289    2
290    2
291    2
292    2
293    2
294    0
295    1
296    0
297    2
298    0
Name: 등급, Length: 299, dtype: int64

In [108]:
df9["RunTime"]

0      137분
1      139분
2      125분
3      133분
4      121분
5      132분
6      109분
7      107분
8      129분
9      141분
10     130분
11     117분
12     139분
13     139분
14      80분
15     136분
16     106분
17      90분
18     119분
19     129분
20     125분
21     106분
22     136분
23     118분
24     151분
25      77분
26     126분
27      97분
28     119분
29     129분
       ... 
269    135분
270     86분
271     87분
272    102분
273     97분
274     86분
275     71분
276    110분
277     83분
278    102분
279     98분
280    120분
281     95분
282    130분
283     60분
284    108분
285    110분
286     85분
287    162분
288     61분
289     96분
290    105분
291    120분
292    104분
293    109분
294     48분
295    107분
296     93분
297    100분
298     84분
Name: RunTime, Length: 299, dtype: object

In [118]:
df9 = df9.drop(["RunTime"], axis=1)

In [119]:
df9.head()

,순번,Movie,개봉일,국적,전국 관객수,장르,NetizenRating,ExpertRating,VideoNumber,국적_한국,day_of_week,등급
0,1,택시운전사,2017-08-02,한국,12099614,드라마,9.04,6.09,20.0,1,1,2
1,2,신과함께-죄와 벌,2017-12-20,한국,8539495,판타지,7.83,5.92,17.0,1,0,1
2,3,공조,2017-01-18,한국,7817446,액션,8.27,5.00,17.0,1,0,2
3,4,스파이더맨: 홈 커밍,2017-07-05,미국,7257973,액션,8.37,7.00,8.0,0,0,1
4,5,범죄도시,2017-10-03,한국,6851297,범죄,9.14,6.00,14.0,1,0,3
